In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import os
import json
from tools.plot_utils import plot_and_save
from tools.data_utils import *
from tools.losses import NPPLoss
from tools.models import Autoencoder
from tools.optimization import EarlyStoppingCallback, evaluate_model
import matplotlib.pyplot as plt
import argparse
import time
from tools.NPmodels import *
from tools.NPtrain import process_batch, NeuralProcessTrainer, evaluate_np

In [9]:
class NPLRFinder:
    def __init__(self, model, optimizer, device='cuda'):
        self.model = model
        self.optimizer = optimizer
        self.device = device
        self.history = {'lr': [], 'loss': []}

    def find_lr(self, train_loader, input_channel, start_lr=1e-6, end_lr=1, num_iter=20,smooth_f=0.05):
        model = self.model.to(self.device)
        optimizer = self.optimizer
        device = self.device
        model.train()

        lr_step = (end_lr / start_lr) ** (1 / num_iter)
        lr = start_lr

        for iteration in range(num_iter):
            optimizer.param_groups[0]['lr'] = lr

            total_loss = 0.0
            for batch in train_loader:
                x_context, y_context, x_target, y_target = process_batch(batch, self.device)
                optimizer.zero_grad()
                p_y_pred, q_target, q_context = model(x_context, y_context, x_target, y_target)
                loss = self._loss(p_y_pred, y_target, q_target, q_context)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()

            avg_loss = total_loss / len(train_loader)
            self.history['lr'].append(lr)
            self.history['loss'].append(avg_loss)

            lr *= lr_step
            
    def plot_lr_finder(self):
        plt.plot(self.history['lr'], self.history['loss'])
        plt.xscale('log')  # Use a logarithmic scale for better visualization
        plt.xlabel('Learning Rate')
        plt.ylabel('Loss')
        plt.title('Learning Rate Finder Curve')
        plt.show()
        
    def find_best_lr(self, skip_start=3, skip_end=3):
        # Find the index of the minimum loss in the specified range
        min_loss_index = skip_start + np.argmin(self.history['loss'][skip_start:-skip_end])
        # Output the learning rate corresponding to the minimum loss
        best_lr = self.history['lr'][min_loss_index]
        return best_lr
    
    def _loss(self, p_y_pred, y_target, q_target, q_context):
        log_likelihood = p_y_pred.log_prob(y_target).mean(dim=0).sum()
        kl = kl_divergence(q_target, q_context).mean(dim=0).sum()
        return -log_likelihood + kl

def custom_collate_fn(batch):
    images = [sample['image'] for sample in batch]
    pins = [sample['pins'] for sample in batch]
    outputs = [sample['outputs'] for sample in batch]

    return {
        'image': torch.stack(images, dim=0),
        'pins': pins,
        'outputs': outputs}

In [10]:
def write_row_to_file(experiment_id, args, config, test_partial_percents, test_losses):
    # Open the file in append mode
    with open(f"./history/{experiment_id}/results.txt", "a") as f:
        # If the file is empty, write the header
        if f.tell() == 0:
            header = "Dataset\tMode\td\tn_pins\tLR\tResults\tMSE of partial percent\tMSE error\n"
            f.write(header)

        # Construct the row
        row = f"{args.dataset}\t{args.mode}\t{args.d}\t{args.n_pins}\t{config['best_lr']}\t{experiment_id}\t{test_partial_percents}\t{test_losses}\n"

        # Write the row to the file
        f.write(row)

In [11]:
def run_pipeline_ci_np(train_loader, val_loader, 
                    test_loader, input_channel, epochs, val_every_epoch, config, device, num_runs=3, print_freq=2):
    test_losses = []
    # partial_percent = config['partial_percent']
    experiment_id = int(time.time())
    best_val_loss_NP = float('inf')
    # config['experiment_id'] = experiment_id
    experiment_id = config['experiment_id']

    r_dim = np_config.r_dim
    h_dim = np_config.h_dim
    z_dim = np_config.z_dim
    lr = config['best_lr']

    # Create storage directory and store the experiment configuration
    if not os.path.exists(f'./history/{experiment_id}'):
        os.makedirs(f'./history/{experiment_id}')
    with open(f"./history/{experiment_id}/config_np.json", "w") as outfile: 
        json.dump(config, outfile)
        
    global_val_loss = float('inf')
    
    for run in range(num_runs):
        count = 0
        GP_test_losses = []
        
        early_stopping = EarlyStoppingCallback(patience=10, min_delta=0.001)
        model = NeuralProcessImg(r_dim, z_dim, h_dim).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        np_trainer = NeuralProcessTrainer(device, model, optimizer, early_stopping, experiment_id, print_freq=print_freq)          

        np_trainer.train(train_loader, val_loader, epochs)
        if np_trainer.best_val_loss <= global_val_loss:
            global_val_loss = np_trainer.best_val_loss 
            torch.save(np_trainer.neural_process.state_dict(), f'./history/{experiment_id}' + f'/best_{args.dataset}_np.pt')
        count += 1

    return experiment_id


# Function to run the pipeline and save data
def run_and_save_pipeline_np(train_loader, val_loader, test_loader, input_channel, epochs, val_every_epoch, config, num_runs, device):
    test_partial_percents = [0.25, 0.5, 0.75, 1]
    test_losses = []
    experiment_id = run_pipeline_ci_np(train_loader, val_loader, 
                    test_loader, input_channel, epochs, val_every_epoch, config, device, num_runs)
    # Run final testing
    model = NeuralProcessImg(r_dim, z_dim, h_dim).to(device)
    # MSE
    model.load_state_dict(torch.load(f'./history/neural_processes/{experiment_id}/best_{args.dataset}_np.pt'))
    for partial_percent in test_partial_percents:
        test_loss = evaluate_np(model, test_loader, device, partial_percent=partial_percent)
        print(f"pp: {partial_percent} MSE loss: {test_loss}")
        test_losses.append(test_loss)    
    write_row_to_file(experiment_id, args, config, test_partial_percents, test_losses)
    print("saved")
    return test_losses, experiment_id

In [12]:
class Args():
    dataset = "PinMNIST"
    n = 1000
    mode = "random"
    d = 3
    n_pins = 100
    r = 3
    partial_percent = 0.8
    # Set your hyperparameters
    epochs = 1000
    batch_size = 64
    learning_rate = 1e-4
    val_every_epoch = 10
    num_runs = 1
    seed = 4
    
class NP_config():
    r_dim = 512
    h_dim = 512
    z_dim = 512
    lr = 4e-5
    epochs = 100

np_config = NP_config()
args = Args()

In [13]:
# Set a random seed for PyTorch
seed = 4  # You can use any integer value as the seed
torch.manual_seed(seed)
# Set a random seed for NumPy (if you're using NumPy operations)
np.random.seed(seed)

 # Choose datasets
dataset = args.dataset 
n = args.n
mode = args.mode
d = args.d
n_pins = args.n_pins
r = args.r
partial_percent = args.partial_percent

# Set your hyperparameters
epochs = args.epochs
batch_size = args.batch_size
learning_rate = args.learning_rate
val_every_epoch = args.val_every_epoch
num_runs = args.num_runs
   
config = {}
# np_config = {"batch_size": 32,
#              "r_dim": 512,
#              "h_dim": 512,
#              "z_dim": 512,
#              "lr": 4e-5,
#              "epochs": 100
#             }

# config = vars(args)
config = {"experiment_id":0}

input_channel = 1 if dataset == "PinMNIST" else 3
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
# if feature_extracted:
#     folder = f"{dataset}_ddpm"
# else:
folder = f"{dataset}"

if dataset == "PinMNIST":
    if mode == "mesh":
        data_folder = f"./data/{folder}/mesh_{d}step_{28}by{28}pixels_{r}radius_{seed}seed"
    else:
        data_folder = f"./data/{folder}/random_fixedTrue_{n_pins}pins_{28}by{28}pixels_{r}radius_{seed}seed"


transform = transforms.Compose([
    ToTensor(),         # Convert to tensor (as you were doing)
    Resize()  # Resize to 100x100
])

transformed_dataset = PinDataset(csv_file=f"{data_folder}/pins.csv",
                                      root_dir=f"./data/{dataset}/images/",
                                      transform=transform)

dataset_size = len(transformed_dataset)
train_size = int(0.7 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size

# Split the dataset into train, validation, and test sets
train_dataset, val_dataset, test_dataset = random_split(
    transformed_dataset, [train_size, val_size, test_size]
)

# Create your DataLoader with the custom_collate_fn
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)

In [ ]:
r_dim, z_dim, h_dim = np_config.r_dim, np_config.z_dim, np_config.h_dim
if (config['experiment_id'] == 0): # Training

    neural_processes = NeuralProcessImg(r_dim, z_dim, h_dim).to(device)
    optimizer = torch.optim.Adam(neural_processes.parameters(), learning_rate)
#     lr_finder_NP =NPLRFinder(neural_processes, optimizer, device=device)
#     lr_finder_NP.find_lr(train_loader,input_channel=input_channel, start_lr=1e-5, end_lr=1, num_iter=20)
#     best_lr_NP = lr_finder_NP.find_best_lr()
#     print(f"Best Learning Rate for NP: {best_lr_NP}")

#     config['best_lr'] = best_lr_NP
    config['best_lr'] = 1e-3
    # Run and save the pipeline data
    test_losses, experiment_id = run_and_save_pipeline_np(train_loader, val_loader, test_loader,\
                                               input_channel, epochs, val_every_epoch, config, num_runs, device)

else: # Testing
    experiment_id = config['experiment_id']
    if not os.path.exists(f'./history/{experiment_id}'):
        raise Exception(f"Could not find experiment with id: {experiment_id}")
    else:
        neural_processes = NeuralProcessImg(r_dim, z_dim, h_dim).to(device)
        try:
            neural_processes.load_state_dict(torch.load(f'./history/neural_processes/{experiment_id}/best_model_NP.pth'))
        except:
            raise Exception("The model you provided does not correspond with the selected architecture. Please revise and try again.")
        
        
        filename = f"test_{folder}_{partial_percent}"
        with open(f"./history/neural_processes/{experiment_id}/{filename}", "w") as f:
            f.write(f"MSE {best_MSE_test_loss}; NPP {best_NPP_test_loss}, {GP_best_NPP_test_loss} (GP)")

Epoch: 0, Avg_loss: 1760.638
Epoch: 0, Val_loss 623.130
Epoch: 1, Avg_loss: 443.932
Epoch: 2, Avg_loss: 365.324
Epoch: 2, Val_loss 357.530
Epoch: 3, Avg_loss: 357.907
Epoch: 4, Avg_loss: 355.449
Epoch: 4, Val_loss 356.767
Epoch: 5, Avg_loss: 353.733
Epoch: 6, Avg_loss: 352.260
Epoch: 6, Val_loss 351.584
Epoch: 7, Avg_loss: 355.794
Epoch: 8, Avg_loss: 355.060
Epoch: 8, Val_loss 356.909
Epoch: 9, Avg_loss: 351.699
Epoch: 10, Avg_loss: 349.229
Epoch: 10, Val_loss 345.372
Epoch: 11, Avg_loss: 343.553
Epoch: 12, Avg_loss: 356.382
Epoch: 12, Val_loss 359.118
Epoch: 13, Avg_loss: 357.218
Epoch: 14, Avg_loss: 346.055
Epoch: 14, Val_loss 345.066
Epoch: 15, Avg_loss: 340.896
Epoch: 16, Avg_loss: 332.455
Epoch: 16, Val_loss 340.386
Epoch: 17, Avg_loss: 321.339
Epoch: 18, Avg_loss: 313.789
Epoch: 18, Val_loss 315.325
Epoch: 19, Avg_loss: 335.893
Epoch: 20, Avg_loss: 326.945
Epoch: 20, Val_loss 320.084
Epoch: 21, Avg_loss: 313.435
Epoch: 22, Avg_loss: 310.062
Epoch: 22, Val_loss 306.067
Epoch: 23, 